In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/SetDueDay/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(3210, 2)


,label,split_text
0,2,你 在 说 什么 ？
1,1,真高 哈哈哈
2,2,怎么样
3,2,听求 不到 ✖ ️ 9
4,1,我 想
5,2,是 的 怎么 了 我 出去 了
6,2,我 还有 事要 做 稍后 联系 我
7,2,你 说 得 太快 而且 不 清楚
8,0,应该 没有 问题
9,1,说实话 老子 是 个 穷人 。


In [3]:
data.split_text.iloc[1]

'真高   哈哈哈'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<3210x7260 sparse matrix of type '<class 'numpy.float64'>'
	with 25257 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.988161993769


In [6]:
print(clf.predict(phrase))

[2 1 2 ..., 0 2 0]


# Logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.965420560748


In [9]:
print(log_r.predict(phrase))

[2 1 2 ..., 0 2 0]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([2, 1, 2, ..., 0, 2, 0])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([2, 1, 2, ..., 0, 2, 0])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.209657
[10]	valid_0's multi_error: 0.187227
[15]	valid_0's multi_error: 0.175701
[20]	valid_0's multi_error: 0.164174
[25]	valid_0's multi_error: 0.154517
[30]	valid_0's multi_error: 0.142368
[35]	valid_0's multi_error: 0.137383
[40]	valid_0's multi_error: 0.13271
[45]	valid_0's multi_error: 0.129283
[50]	valid_0's multi_error: 0.123676
[55]	valid_0's multi_error: 0.120561
[60]	valid_0's multi_error: 0.118692
[65]	valid_0's multi_error: 0.115888
[70]	valid_0's multi_error: 0.114019
[75]	valid_0's multi_error: 0.113084
[80]	valid_0's multi_error: 0.111526
[85]	valid_0's multi_error: 0.109969
[90]	valid_0's multi_error: 0.11028
[95]	valid_0's multi_error: 0.107788
[100]	valid_0's multi_error: 0.1081
[105]	valid_0's multi_error: 0.107477
[110]	valid_0's multi_error: 0.106854
[115]	valid_0's multi_error: 0.106542
[120]	valid_0's multi_error: 0.105607
[125]	valid_0's multi_error: 0.103738
[130]	valid

In [17]:
print(lgbm_model.predict(phrase))

[[ 0.00257452  0.00213926  0.99760379]
 [ 0.39121786  0.19567043  0.34040748]
 [ 0.39121786  0.19567043  0.34040748]
 ..., 
 [ 0.93145699  0.04999604  0.03302054]
 [ 0.00266496  0.00160718  0.99809674]
 [ 0.90423273  0.06132981  0.10268657]]


# Save

In [18]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/SetDueDay/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/SetDueDay/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/SetDueDay/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/SetDueDay/Lgbm.pickle", "wb"))

# test

In [23]:
test = phrase_vectorizer.transform(['不 可能'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])

In [24]:
clf.predict_proba(test) # linear svc

array([[ 0.00531646,  0.98382451,  0.01085902]])

In [25]:
log_r.predict_proba(test) # logistic

array([[ 0.04516838,  0.86240357,  0.09242805]])

In [26]:
lgbm_model.predict(test) # light gbm

array([[ 0.00508463,  0.97758581,  0.01437897]])

# Labeling

In [27]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.983824511806


# Save the whole thing

In [28]:
# import jieba
# import numpy as np

# class SetDueDay:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred<threshold):
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [29]:
%load_ext autoreload
%autoreload 2

In [30]:
from SetDueDay_py import SetDueDay
ifk = SetDueDay(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer, jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/SetDueDay/SetDueDay.pickle", "wb"))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.460 seconds.
Prefix dict has been built succesfully.
